### Instalaciones iniciales

In [7]:
#%pip install pygame tensorflow
%pip install tensorflow

  Using cached tensorflow-2.15.0-cp310-cp310-win_amd64.whl (2.1 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: C:\Users\PC\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# El Codigo para correr 'Dinosaur Game'

In [8]:
import tensorflow as tf
from tensorflow import keras
import random

ImportError: cannot import name 'keras' from 'tensorflow' (unknown location)

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(7,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3)
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
target_model = tf.keras.models.clone_model(model)


target_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
target_model.set_weights(model.get_weights())

AttributeError: module 'tensorflow' has no attribute 'keras'

In [ ]:
INIT_REPLAY_MEM_SIZE = 5000
REPLAY_MEMORY_SIZE = 45000
MIN_REPLAY_MEMORY_SIZE = 1000
MINIBATCH_SIZE = 64
DISCOUNT = 0.95
UPDATE_TARGET_THRESH = 5
EPSILON_INIT = 0.45
EPSILON_DECAY = 0.997
NUM_EPISODES = 2000
MIN_EPSILON = 0.05

In [ ]:
def train(model, target_model, replay_memory, target_update_counter):
    # Verificar si hay suficientes datos para entrenar
    if len(replay_memory) < MIN_REPLAY_MEMORY_SIZE:
        return target_update_counter

    # Tomar un minibatch al azar de la memoria de replay
    minibatch = random.sample(replay_memory, MINIBATCH_SIZE)

    # Separar los componentes del minibatch
    current_states = np.array([transition[0] for transition in minibatch])
    current_qs_list = model.predict(current_states)
    new_current_states = np.array([transition[3] for transition in minibatch])
    future_qs_list = target_model.predict(new_current_states)

    X = []
    y = []

    # Recorrer el minibatch y actualizar los Q-values
    for index, (current_state, action, reward, new_current_state, done) in enumerate(minibatch):
        if not done:
            max_future_q = np.max(future_qs_list[index])
            new_q = reward + DISCOUNT * max_future_q
        else:
            new_q = reward

        current_qs = current_qs_list[index]
        current_qs[action] = new_q

        X.append(current_state)
        y.append(current_qs)

    # Entrenar el modelo principal
    model.fit(np.array(X), np.array(y), batch_size=MINIBATCH_SIZE, shuffle=False, verbose=0)

    # Actualizar el modelo objetivo si es necesario
    if target_update_counter > UPDATE_TARGET_THRESH:
        target_model.set_weights(model.get_weights())
        target_update_counter = 0

    return target_update_counter

In [ ]:
def get_game_state(player, obstacles, game_speed):
    if obstacles:
        # Ejemplo de cómo podrías estructurar el estado:
        # - Distancia al próximo obstáculo
        # - Tipo de obstáculo (p.ej., cactus pequeño, cactus grande, pájaro)
        # - Velocidad del juego
        # - Y-coordinate del dinosaurio
        # Adapta esto según tus necesidades y estructura del juego
        obstacle = obstacles[0]
        return np.array([
            obstacle.rect.x - player.dino_rect.x,  # Distancia al obstáculo
            1 if obstacle.type == 'small_cactus' else 0,  # Tipo de obstáculo (one-hot encoded)
            1 if obstacle.type == 'large_cactus' else 0,
            1 if obstacle.type == 'bird' else 0,
            game_speed,  # Velocidad del juego
            player.dino_rect.y,  # Y-coordinate del dinosaurio
            0  # Puedes agregar más características si lo consideras necesario
        ])
    else:
        return np.zeros((7,))  # Estado vacío si no hay obstáculos

def perform_action(player, action):
    if action == 0:
        # Saltar
        player.jump()
    elif action == 1:
        # Agacharse
        player.duck()
    elif action == 2:
        # Correr (mantenerse en el suelo)
        pass  # No es necesario hacer nada para correr

def get_reward(player, obstacles):
    if not obstacles:
        return 0.1  # Recompensa pequeña por seguir vivo sin obstáculos
    obstacle = obstacles[0]
    if player.dino_rect.colliderect(obstacle.rect):
        return -1  # Recompensa negativa por chocar
    else:
        return 0.8  # Recompensa por evitar el obstáculo
    
def check_if_game_over(obstacles, player):
    if not obstacles:
        return False
    obstacle = obstacles[0]
    return player.dino_rect.colliderect(obstacle.rect)  # El juego termina si hay una colisión

In [ ]:
import pygame
import os
import numpy as np
import random
pygame.init()

# Global Constants
SCREEN_HEIGHT = 600
SCREEN_WIDTH = 1100
SCREEN = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

RUNNING = [pygame.image.load(os.path.join("Assets/Dino", "DinoRun1.png")),
           pygame.image.load(os.path.join("Assets/Dino", "DinoRun2.png"))]
JUMPING = pygame.image.load(os.path.join("Assets/Dino", "DinoJump.png"))
DUCKING = [pygame.image.load(os.path.join("Assets/Dino", "DinoDuck1.png")),
           pygame.image.load(os.path.join("Assets/Dino", "DinoDuck2.png"))]

SMALL_CACTUS = [pygame.image.load(os.path.join("Assets/Cactus", "SmallCactus1.png")),
                pygame.image.load(os.path.join("Assets/Cactus", "SmallCactus2.png")),
                pygame.image.load(os.path.join("Assets/Cactus", "SmallCactus3.png"))]
LARGE_CACTUS = [pygame.image.load(os.path.join("Assets/Cactus", "LargeCactus1.png")),
                pygame.image.load(os.path.join("Assets/Cactus", "LargeCactus2.png")),
                pygame.image.load(os.path.join("Assets/Cactus", "LargeCactus3.png"))]

BIRD = [pygame.image.load(os.path.join("Assets/Bird", "Bird1.png")),
        pygame.image.load(os.path.join("Assets/Bird", "Bird2.png"))]

CLOUD = pygame.image.load(os.path.join("Assets/Other", "Cloud.png"))

BG = pygame.image.load(os.path.join("Assets/Other", "Track.png"))


class Dinosaur:
    X_POS = 80
    Y_POS = 310
    Y_POS_DUCK = 340
    JUMP_VEL = 8.5

    def __init__(self):
        self.duck_img = DUCKING
        self.run_img = RUNNING
        self.jump_img = JUMPING

        self.dino_duck = False
        self.dino_run = True
        self.dino_jump = False

        self.step_index = 0
        self.jump_vel = self.JUMP_VEL
        self.image = self.run_img[0]
        self.dino_rect = self.image.get_rect()
        self.dino_rect.x = self.X_POS
        self.dino_rect.y = self.Y_POS


        
    def update(self, action=None):
        if action == 0:  # Saltar
            self.dino_duck = False
            self.dino_run = False
            self.dino_jump = True
        elif action == 1:  # Agacharse
            self.dino_duck = True
            self.dino_run = False
            self.dino_jump = False
        elif action == 2:  # Correr
            self.dino_duck = False
            self.dino_run = True
            self.dino_jump = False

        if self.dino_jump:
            self.jump()
        elif self.dino_duck:
            self.duck()
        else:
            self.run()
            
    def duck(self):
        self.step_index += 1
        if self.step_index >= len(self.duck_img) * 5:  # Asegúrate de que el índice no exceda el tamaño de la lista
            self.step_index = 0

        self.image = self.duck_img[self.step_index // 5]
        self.dino_rect = self.image.get_rect()
        self.dino_rect.x = self.X_POS
        self.dino_rect.y = self.Y_POS_DUCK
        
        
    def run(self):
        self.step_index += 1
        if self.step_index >= len(self.run_img) * 5:  # Multiplicado por 5 porque es el divisor usado
            self.step_index = 0

        self.image = self.run_img[self.step_index // 5]
        self.dino_rect = self.image.get_rect()
        self.dino_rect.x = self.X_POS
        self.dino_rect.y = self.Y_POS
        
    def jump(self):
        self.image = self.jump_img
        if self.dino_jump:
            self.dino_rect.y -= self.jump_vel * 4
            self.jump_vel -= 0.8
        if self.jump_vel < - self.JUMP_VEL:
            self.dino_jump = False
            self.jump_vel = self.JUMP_VEL
            self.dino_rect.y = 380

    def draw(self, SCREEN):
        SCREEN.blit(self.image, (self.dino_rect.x, self.dino_rect.y))


class Cloud:
    def __init__(self):
        self.x = SCREEN_WIDTH + random.randint(800, 1000)
        self.y = random.randint(50, 100)
        self.image = CLOUD
        self.width = self.image.get_width()

    def update(self):
        self.x -= game_speed
        if self.x < -self.width:
            self.x = SCREEN_WIDTH + random.randint(2500, 3000)
            self.y = random.randint(50, 100)

    def draw(self, SCREEN):
        SCREEN.blit(self.image, (self.x, self.y))


class Obstacle:
    def __init__(self, image, type):
        self.image = image
        self.type = type
        self.rect = self.image[self.type].get_rect()
        self.rect.x = SCREEN_WIDTH

    def update(self):
        self.rect.x -= game_speed
        if self.rect.x < -self.rect.width:
            obstacles.pop()

    def draw(self, SCREEN):
        SCREEN.blit(self.image[self.type], self.rect)


class SmallCactus(Obstacle):
    def __init__(self, image):
        self.type = random.randint(0, 2)
        super().__init__(image, self.type)
        self.rect.y = 325


class LargeCactus(Obstacle):
    def __init__(self, image):
        self.type = random.randint(0, 2)
        super().__init__(image, self.type)
        self.rect.y = 300


class Bird(Obstacle):
    def __init__(self, image):
        self.type = 0
        super().__init__(image, self.type)
        self.rect.y = 250
        self.index = 0

    def draw(self, SCREEN):
        if self.index >= 9:
            self.index = 0
        SCREEN.blit(self.image[self.index//5], self.rect)
        self.index += 1

In [ ]:
def main(model, target_model):
    global game_speed, x_pos_bg, y_pos_bg, points, obstacles
    run = True
    clock = pygame.time.Clock()
    player = Dinosaur()
    cloud = Cloud()
    game_speed = 20
    x_pos_bg = 0
    y_pos_bg = 380
    points = 0
    obstacles = []
    death_count = 0
    replay_memory = []  # Almacenar experiencias para el entrenamiento
    target_update_counter = 0  # Contador para actualizar el target_model
    epsilon = MIN_EPSILON

    def score():
        global points, game_speed
        points += 1
        if points % 100 == 0:
            game_speed += 1

        font = pygame.font.Font('freesansbold.ttf', 20)
        text = font.render("Points: " + str(points), True, (0, 0, 0))
        textRect = text.get_rect()
        textRect.center = (1000, 40)
        SCREEN.blit(text, textRect)

    def background():
        global x_pos_bg, y_pos_bg
        image_width = BG.get_width()
        SCREEN.blit(BG, (x_pos_bg, y_pos_bg))
        SCREEN.blit(BG, (image_width + x_pos_bg, y_pos_bg))
        if x_pos_bg <= -image_width:
            x_pos_bg = 0
        x_pos_bg -= game_speed

    while run:
        SCREEN.fill((255, 255, 255))

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False

        # Obtener el estado actual del juego
        current_state = get_game_state(player, obstacles, game_speed)

        
        if np.random.rand() < epsilon:
            action = np.random.randint(0, 3)  # Random action: 0, 1, or 2
        else:
            action = np.argmax(model.predict(current_state.reshape(-1, 7))[0])

        epsilon = max(MIN_EPSILON, epsilon * EPSILON_DECAY)

        print("Action chosen:", action)  # Imprime la acción para depurar
        player.update(action)
        
        
        # Agregar nuevos obstáculos
        if len(obstacles) == 0 or (obstacles and obstacles[-1].rect.x < SCREEN_WIDTH - 300):
            obstacle_type = random.choice(['small_cactus', 'large_cactus', 'bird'])
            if obstacle_type == 'small_cactus':
                obstacles.append(SmallCactus(SMALL_CACTUS))
            elif obstacle_type == 'large_cactus':
                obstacles.append(LargeCactus(LARGE_CACTUS))
            elif obstacle_type == 'bird':
                obstacles.append(Bird(BIRD))

        # Actualizar y dibujar nubes y obstáculos
        cloud.update()
        cloud.draw(SCREEN)
        for obstacle in obstacles:
            obstacle.update()
            obstacle.draw(SCREEN)

        # Actualizar y dibujar el dinosaurio
        player.draw(SCREEN)

        # Verificar colisiones
        if any(player.dino_rect.colliderect(obstacle.rect) for obstacle in obstacles):
            run = False

        # Dibujar el fondo y el puntaje
        background()
        score()

        # Actualizar la pantalla y mantener la velocidad del juego
        pygame.display.update()
        clock.tick(30)

        # Obtener el nuevo estado y la recompensa después de realizar la acción
        new_state = get_game_state(player, obstacles, game_speed)
        reward = get_reward(player, obstacles)
        done = not run

        # Guardar en memoria de replay
        replay_memory.append((current_state, action, reward, new_state, done))

        # Entrenar la red neuronal
        if len(replay_memory) > MIN_REPLAY_MEMORY_SIZE:
            target_update_counter = train(model, target_model, replay_memory, target_update_counter)

        # Reiniciar el juego si es necesario
        if not run:
            run = True
            player = Dinosaur()
            cloud = Cloud()
            obstacles = []
            game_speed = 20
            points = 0  # Reiniciar puntos

In [ ]:
def menu(death_count):
    global points
    run = True
    while run:
        SCREEN.fill((255, 255, 255))
        font = pygame.font.Font('freesansbold.ttf', 30)

        if death_count == 0:
            text = font.render("Press any Key to Start", True, (0, 0, 0))
        elif death_count > 0:
            text = font.render("Press any Key to Restart", True, (0, 0, 0))
            score = font.render("Your Score: " + str(points), True, (0, 0, 0))
            scoreRect = score.get_rect()
            scoreRect.center = (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 + 50)
            SCREEN.blit(score, scoreRect)
        textRect = text.get_rect()
        textRect.center = (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2)
        SCREEN.blit(text, textRect)
        SCREEN.blit(RUNNING[0], (SCREEN_WIDTH // 2 - 20, SCREEN_HEIGHT // 2 - 140))
        pygame.display.update()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
            if event.type == pygame.KEYDOWN:
                main(model, target_model)


menu(death_count=0)